
# Movie Reviews Prediction

## Logistic regression with Sklearn

### Import data and library

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Import pandas, numpy and the dataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [4]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')
  

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [5]:
!pip install contractions
!pip install autocorrect


### Read data and clean up

In [0]:
# Read file 
data = pd.read_csv('/content/drive/My Drive/FTMLE - Tonga/Data/movie_review.csv', encoding='utf-8', sep='\t')
data = data

In [7]:
data.head(10)

,id,review,sentiment
0,5814_8,With all this stuff going down at the moment w...,1
1,2381_9,"\The Classic War of the Worlds\"" by Timothy Hi...",1
2,7759_3,The film starts with a manager (Nicholas Bell)...,0
3,3630_4,It must be assumed that those who praised this...,0
4,9495_8,Superbly trashy and wondrously unpretentious 8...,1
5,8196_8,I dont know why people think this is such a ba...,1
6,7166_2,"This movie could have been very good, but come...",0
7,319_1,"A friend of mine bought this film for Â£1, and...",0
8,8713_10,<br /><br />This movie is full of references. ...,1
9,2486_3,"What happens when an army of wetbacks, towelhe...",0


In [0]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 2000)
    pd.set_option('display.float_format', '{:20,.2f}'.format)
    pd.set_option('display.max_colwidth', -1)
    print(x)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.float_format')
    pd.reset_option('display.max_colwidth')


In [0]:
# Drop id
data.drop(labels=['id'], axis=1, inplace=True)

In [10]:
test_text = data.iloc[data['review'].str.len().idxmax()].iloc[0]
print_full(test_text)

Match 1: Tag Team Table Match Bubba Ray and Spike Dudley vs Eddie Guerrero and Chris Benoit Bubba Ray and Spike Dudley started things off with a Tag Team Table Match against Eddie Guerrero and Chris Benoit. According to the rules of the match, both opponents have to go through tables in order to get the win. Benoit and Guerrero heated up early on by taking turns hammering first Spike and then Bubba Ray. A German suplex by Benoit to Bubba took the wind out of the Dudley brother. Spike tried to help his brother, but the referee restrained him while Benoit and Guerrero ganged up on him in the corner. With Benoit stomping away on Bubba, Guerrero set up a table outside. Spike dashed into the ring and somersaulted over the top rope onto Guerrero on the outside! After recovering and taking care of Spike, Guerrero slipped a table into the ring and helped the Wolverine set it up. The tandem then set up for a double superplex from the middle rope which would have put Bubba through the table, but

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


### Text Processing

In [0]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

stop = stopwords.words('english')

#### Removing special characters and "trash"


In [12]:
!pip install beautifulsoup4
!pip install -U spacy
!python -m spacy download en_core_web_sm

Requirement already up-to-date: spacy in /usr/local/lib/python3.6/dist-packages (2.2.4)
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [0]:
import re
import contractions
import string
from bs4 import BeautifulSoup
import unicodedata

def preprocessor(text):
    """ Return a cleaned version of text
    """
    # Remove html tags
    text = strip_html_tags(text)
    # Remove accents 
    text = remove_accented_chars(text)
    # remove extra newlines
    text = re.sub(r'[\r|\n|\r\n]+', ' ',text)

    # Expanding contractions words
    # text = contractions.fix(text)
    
    # Save emoticons for later appending
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)

    # text.replace('...',' ')
    # text.replace('-',' ')

    # Remove accents 
    text = remove_accented_chars(text)

    # # insert spaces between special characters to isolate them    
    # special_char_pattern = re.compile(r'([{.(-)!}])')
    # text = remove_special_characters(text)  

    # remove extra whitespace
    text = re.sub(' +', ' ', text)  

    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))

    return text

def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def strip_html_tags(text):
    soup = BeautifulSoup(text, "lxml")
    stripped_text = soup.get_text()
    return stripped_text

# # Removing Special Characters
def remove_special_characters(text):
    text = re.sub('[^a-zA-Z0-9\s]', '', text)
    return text


In [14]:
%%time
test = data['review'].sample(10)
print_full(test)
print_full(test.apply(preprocessor))

7753     Hardly a masterpiece. Not so well written. Beautiful cinematography i think not. This movie wasn't too terrible but it wasn't that much better than average. The main story dealing with highly immoral teens should have focused more on the forbidden romance and why this was... should have really gotten into it instead of scraping the surface with basically \because mom says we can't.\" Some parts should have been dropped altogether or reworked to have more importance to the plight of the two main characters. Couple times i was wondering if the writer/director was a fan of George Lucas' classic American Graffiti. Not that it's wrong to be a fan of that movie but to make your movie at times look like that, i mean come on! Worst part of this was that Madchen Amick had such a small part, i mean double come on!! She was the only one, in one or two lines, who actually tried a southern accent. (Take a good listen, it was there even though her character was from California! DOH!!) Maybe

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


#### Prepare tokenizer

In [0]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
import string
from sklearn.base import TransformerMixin
from nltk.tokenize.toktok import ToktokTokenizer


punctuations = string.punctuation
tokenizer = ToktokTokenizer()
stop_words = spacy.lang.en.stop_words.STOP_WORDS
parser = spacy.load("en_core_web_sm")

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    sentence = remove_stopwords(sentence)
    # Keep meaningfull words
    # meaningfull_words = get_meaningfull_words(sentence)
    # for word in meaningfull_words:
    #   sentence = sentence.replace(word,'')
    # Creating our token object, which is used to create documents with linguistic annotations.
    tokens = parser(sentence)
    # # Lemmatizing each token and converting each token into lowercase
    tokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in tokens ]
    # tokens.extend(meaningfull_words)

   # return preprocessed list of tokens
    return tokens


def spacy_filter_meaningfull_word_tokenizer(sentence):
    sentence = remove_stopwords(sentence)
    # Keep meaningfull words
    meaningfull_words = get_meaningfull_words(sentence)
    for word in meaningfull_words:
      sentence = sentence.replace(word,'')
    # Creating our token object, which is used to create documents with linguistic annotations.
    tokens = parser(sentence)
    # # Lemmatizing each token and converting each token into lowercase
    tokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in tokens ]
    tokens.extend(meaningfull_words)

   # return preprocessed list of tokens
    return tokens

def get_meaningfull_words(sentence):
  parser = spacy.load("en_core_web_sm")
  tokens = parser(sentence)
  return [ent.text for ent in tokens.ents]



# # Removing Stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stop_words]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text



# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()


In [16]:
%%time
test = data['review'].sample(10)
print_full(test)
print_full(test.apply(preprocessor).apply(spacy_tokenizer))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


15186    \Three Daring Daughters\" is a sickly sweet, rose-colored look at divorce, remarriage, and single-parent living. Obviously, social issues and economic difficulty have no place in the picture perfect life of a single parent mother who feels exhausted, takes a cruise, and then dates and marries a band conductor. Even when the \"its just a movie\" phrase excuses the script from addressing real-life problems, 'Daughters' suffers from too many incoherent high-note songs, children whose personalities are not based on real children and band leader Hose Iturbi playing himself. Isn't it bizarre that any real person would star in a film in which their supposed real self gets married? <br /><br />Admittedly, this movie was released in the nineteen forties. Only a love for old style Hollywood romance and comedy could make 'Daughters' a tolerable film."                                                                                                                                           

### Split train and test sample

In [0]:
# split the dataset in train and test
# Your code here
from sklearn.model_selection import train_test_split
X = data['review']
y = data['sentiment']
X_train,X_test,y_train, y_test = train_test_split(X,y,test_size=0.2)


In [18]:
print(X.shape)
print(y.shape)
print(X_train.shape)
print(y_train.shape)

(22500,)
(22500,)
(18000,)
(18000,)


### Classification



In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.externals.joblib import Memory
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer,preprocessor=preprocessor,
                               ngram_range=(1,1), max_df = 0.5, min_df = 1,sublinear_tf=True)

log_reg = LogisticRegression(n_jobs=-1,random_state=0,C=3.5938136638046276)
tfidf_pipe =  Pipeline([("cleaner", predictors()),
                 ('vect', tfidf_vector),
                 ('clf', log_reg)],memory = Memory(location='/content/cache'))


In [29]:
%%time
# Train model

# model generation
tfidf_pipe.fit(X_train,y_train)
# Predict

tfidf_pipe_predictions = tfidf_pipe.predict(X_test)
# Evaluate
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, tfidf_pipe_predictions))
print("Logistic Regression Precision:",metrics.precision_score(y_test, tfidf_pipe_predictions))
print("Logistic Regression Recall:",metrics.recall_score(y_test, tfidf_pipe_predictions))

________________________________________________________________________________
[Memory] Calling sklearn.pipeline._fit_transform_one...
_fit_transform_one(<__main__.predictors object at 0x7fb149ad55f8>, 10485    I was always a big fan of this movie, first of...
15329    Considering the big name cast and lavish produ...
2294     Part II or formerly known as GUERILLA, is also...
6986     Unlike other commenters who have commented on ...
12690    Redundant, but again the case. If you enjoy th...
                               ...                        
5265     I really enjoyed \Doctor Mordrid\". This is a ...
19111    Here's a horror version of PRISCILLA: QUEEN OF...
14721    There are movies, and there are films. Movies ...
1662     I love killer Insects movies they are great fu...
18206    This project was originally conceived as the m...
Name: review, Length: 18000, dtype: object, 
10485    1
15329    1
2294     1
6986     1
12690    0
        ..
5265     1
19111    0
14721    1
166

/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:315: UserWarning: Persisting input arguments took 7.12s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  **fit_params_steps[name])


Logistic Regression Accuracy: 0.8946666666666667
Logistic Regression Precision: 0.8860149057430952
Logistic Regression Recall: 0.9042505592841164
CPU times: user 9min 27s, sys: 1.82 s, total: 9min 29s
Wall time: 9min 31s


In [1]:
tfidf_filter_vector = TfidfVectorizer(tokenizer = spacy_filter_meaningfull_word_tokenizer,preprocessor=preprocessor,
                               ngram_range=(1,1), max_df = 0.5, min_df = 1,sublinear_tf=True)

log_reg = LogisticRegression(n_jobs=-1,random_state=0,C=3.5938136638046276)
tfidf_filter_pipe =  Pipeline([("cleaner", predictors()),
                 ('vect', tfidf_filter_vector),
                 ('clf', log_reg)],memory = Memory(location='/content/cache'))


NameError: ignored

In [0]:
%%time
# Train model

# model generation
tfidf_filter_pipe.fit(X_train,y_train)
# Predict

tfidf_filter_pipe_predictions = tfidf_filter_pipe.predict(X_test)
# Evaluate
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, tfidf_filter_pipe_predictions))
print("Logistic Regression Precision:",metrics.precision_score(y_test, tfidf_filter_pipe_predictions))
print("Logistic Regression Recall:",metrics.recall_score(y_test, tfidf_filter_pipe_predictions))

________________________________________________________________________________
[Memory] Calling sklearn.pipeline._fit_transform_one...
_fit_transform_one(<__main__.predictors object at 0x7f4563867940>, 19716    The Little Mermaid is one of my absolute favor...
5938     \Whipped\" is one of the most awful films of a...
15650    Dennis Hopper is without a doubt one of the fi...
5065     Madhur Bhandarkar goes all out to touch upon t...
2850     A bondage, humiliation, S&M show, and not much...
                               ...                        
2387     the Germans all stand out in the open and get ...
16957    I expected a comedy like the \Big Mama\" movie...
21559    As soon as I heard about this film I knew I ha...
3262     A plot that fizzled and reeked of irreconcilab...
19884    I am a guy, who loves guy movies... I was look...
Name: review, Length: 18000, dtype: object, 
19716    0
5938     0
15650    0
5065     1
2850     0
        ..
2387     0
16957    0
21559    1
326

In [0]:
# Create hyperparameter options
# Create regularization hyperparameter space
C = np.logspace(-5,5,10)
ngram_range = [(1,1),(1,2),(1,3),(2,2),(2,3)]
max_df = (0.25,0.5,0.75,1)
min_df = (0.25,0.5,0.75,1)
binary = (True,False)
norm = ('l1','l2')
use_idf = (True,False)
smooth_idf = (True,False)
sublinear_tf = (True,False)

param_grid = {
  'clf__C': C,
  # 'vect__ngram_range': ngram_range,
  # 'vect__max_df': max_df,'vect__min_df': min_df
  # 'vect__binary': binary, 'vect__norm': norm,
  # 'vect__use_idf': use_idf, 'vect__smooth_idf': smooth_idf,
  # 'vect__sublinear_tf':  sublinear_tf
}
gs = GridSearchCV(tfidf_filter_pipe, param_grid,n_jobs=-1 ,cv=5, verbose=1)


In [0]:
from sklearn.externals.joblib import parallel_backend

# model generation
with parallel_backend('multiprocessing'):
  gs.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 2 concurrent workers.


________________________________________________________________________________
[Memory] Calling sklearn.pipeline._fit_transform_one...
_fit_transform_one(<__main__.predictors object at 0x7fb151b18358>, 9370     I might not be a huge admirer of the original ...
16239    Watch the Original with the same title from 19...
10757    I rarely write a negative review for this site...
6969     This is the best movie I've seen since White a...
14159    I started to watch this movie expecting nothin...
                               ...                        
5265     I really enjoyed \Doctor Mordrid\". This is a ...
19111    Here's a horror version of PRISCILLA: QUEEN OF...
14721    There are movies, and there are films. Movies ...
1662     I love killer Insects movies they are great fu...
18206    This project was originally conceived as the m...
Name: review, Length: 14400, dtype: object, 
9370     0
16239    0
10757    0
6969     1
14159    1
        ..
5265     1
19111    0
14721    1
166

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-33-599e14f4a495>", line 5, in <module>
    gs.fit(X_train,y_train)
  File "/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py", line 710, in fit
    self._run_search(evaluate_candidates)
  File "/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py", line 1151, in _run_search
    evaluate_candidates(ParameterGrid(self.param_grid))
  File "/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py", line 689, in evaluate_candidates
    cv.split(X, y, groups)))
  File "/usr/local/lib/python3.6/dist-packages/joblib/parallel.py", line 1017, in __call__
    self.retrieve()
  File "/usr/local/lib/python3.6/dist-packages/joblib/parallel.py", line 909, in retrieve
    self._output.extend(job.get(timeout=self.timeout))
  Fil

KeyboardInterrupt: ignored

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/zmq/eventloop/zmqstream.py", line 438, in _run_callback
    callback(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/tornado/stack_context.py", line 277, in null_wrapper
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelbase.py", line 283, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelbase.py", line 233, in dispatch_shell
    handler(stream, idents, msg)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelbase.py", line 421, in execute_request
    self._abort_queues()
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelbase.py", line 636, in _abort_queues
    self._abort_queue(stream)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelbase.py", line 657, in _abort_queue
    sel

In [0]:
%%time
# Predicting with a test dataset
gs_predictions = gs.predict(X_test)

CPU times: user 24.3 s, sys: 100 ms, total: 24.4 s
Wall time: 24.5 s


In [0]:
from sklearn import metrics


# View best hyperparameters
print('Best params:', gs.best_params_)
# print('Best C:', gs.best_estimator_.get_params()['C'])
# print('Best dual:', gs.best_estimator_.get_params()['dual'])
# print('Best fit_intercept:', gs.best_estimator_.get_params()['fit_intercept'])
# print('Best class_weight:', gs.best_estimator_.get_params()['class_weight'])
# print('Best random_state:', gs.best_estimator_.get_params()['random_state'])
# print('Best solver:', gs.best_estimator_.get_params()['solver'])


# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, gs_predictions))
print("Logistic Regression Precision:",metrics.precision_score(y_test, gs_predictions))
print("Logistic Regression Recall:",metrics.recall_score(y_test, gs_predictions))


Best params: {'clf__C': 3.5938136638046276}
Logistic Regression Accuracy: 0.8805387205387205
Logistic Regression Precision: 0.8723854911305269
Logistic Regression Recall: 0.8905405405405405


In [0]:
tweets = [
    "This is really bad",
    "I love this!",
    ":)",
]

preds = clf.predict_proba(tweets)

for i in range(len(tweets)):
    print(f'{tweets[i]} --> Positive, Negative = {preds[i]}')

#### Save Model

In [0]:
import pickle
import os

pickle.dump(tfidf_pipe, open(os.path.join('models', 'movie_review.pkl'), 'wb'), protocol=4)

In [0]:
# Evaluate model
evaluation_data = pd.read_csv('/content/drive/My Drive/FTMLE - Tonga/Data/movie_review_evaluation.csv',sep='\t')
X_evaluation = evaluation_data['review']
y_pred = tfidf_pipe.predict(X_evaluation)
pred_sentiment = pd.DataFrame(y_pred,columns=['sentiment'])
evaluation_data = pd.concat([X_evaluation,pred_sentiment], axis=1)
evaluation_data.to_csv('/content/models/phuc.tran.csv')

### Addition